In [1]:
!nvidia-smi

Sun Oct 18 15:43:55 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 455.23.05    Driver Version: 418.67       CUDA Version: 10.1     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   40C    P8     9W /  70W |      0MiB / 15079MiB |      0%      Default |
|                               |                      |                 ERR! |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
%cd /content/gdrive/My Drive/fast_ds/thesis/experimentation/Segmentation/modification_testing
!ls

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/My Drive/fast_ds/thesis/experimentation/Segmentation/modification_testing
ResUNetplusplus


In [3]:
%cd ./ResUNetplusplus
!ls 

/content/gdrive/My Drive/fast_ds/thesis/experimentation/Segmentation/modification_testing/ResUNetplusplus
data			m_resunet.py	  Resunet++.png
data_generator.py	new_data	  resunet.py
execution_script.ipynb	process_image.py  run_miouTesting.py
files			__pycache__	  run_modified.py
infer.py		README.md	  run.py
metrics.py		result		  run_sample.py
m_resunet_modified.py	results.png	  unet.py


In [15]:
# %pycat process_image.py
# %%writefile process_image.py
!python3 process_image.py #- to augment training dataset.
# !python3 data_generator.py
# Total Size:  1000
# Training Size:  800
# Validation Size:  100
# Testing Size:  100
# 100% 100/100 [00:04<00:00, 24.42it/s]
# 100% 100/100 [00:04<00:00, 22.95it/s]
# 100% 800/800 [53:22<00:00,  4.00s/it]

Total Size:  1000
Training Size:  800
Validation Size:  100
Testing Size:  100
100% 100/100 [00:03<00:00, 29.04it/s]
100% 100/100 [00:03<00:00, 27.02it/s]
100% 800/800 [43:36<00:00,  3.27s/it]


In [16]:
# %pycat run_modified.py
# %%writefile run_modified.py
!python3 run_modified.py

2020-10-18 09:44:19.594875: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
train steps: 3100 valid_steps 12
2020-10-18 09:44:21.612422: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-18 09:44:21.616856: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-18 09:44:21.617394: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla T4 computeCapability: 7.5
coreClock: 1.59GHz coreCount: 40 deviceMemorySize: 14.73GiB deviceMemoryBandwidth: 298.08GiB/s
2020-10-18 09:44:21.617438: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-18 09:44:21.619161: I tensorflow

In [17]:
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import CustomObjectScope
from data_generator import *
from metrics import dice_coef, dice_loss, miou_coef, miou_loss

def mask_to_3d(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

if __name__ == "__main__":
    model_path = "files/daspp_relu_miou.h5"
    save_path = "result"
    test_path = "new_data/Kvasir-SEG/test/"

    image_size = 256
    batch_size = 1

    test_image_paths = glob(os.path.join(test_path, "images", "*"))
    test_mask_paths = glob(os.path.join(test_path, "masks", "*"))
    test_image_paths.sort()
    test_mask_paths.sort()

    ## Create result folder
    try:
        os.mkdir(save_path)
    except:
        pass

    ## Model
    with CustomObjectScope({'dice_loss': dice_loss, 'dice_coef': dice_coef,'miou_loss':miou_loss,'miou_coef':miou_coef}):
        model = load_model(model_path)

    ## Test
    print("Test Result: ")
    test_steps = len(test_image_paths)//batch_size
    test_gen = DataGen(image_size, test_image_paths, test_mask_paths, batch_size=batch_size)
    model.evaluate_generator(test_gen, steps=test_steps, verbose=1)

    ## Generating the result
    for i, path in tqdm(enumerate(test_image_paths), total=len(test_image_paths)):
        image = parse_image(test_image_paths[i], image_size)
        mask = parse_mask(test_mask_paths[i], image_size)

        predict_mask = model.predict(np.expand_dims(image, axis=0))[0]
        predict_mask = (predict_mask > 0.5) * 255.0

        sep_line = np.ones((image_size, 10, 3)) * 255

        mask = mask_to_3d(mask)
        predict_mask = mask_to_3d(predict_mask)

        all_images = [image * 255, sep_line, mask * 255, sep_line, predict_mask]
        cv2.imwrite(f"{save_path}/{i}.png", np.concatenate(all_images, axis=1))

Test Result: 
Instructions for updating:
Please use Model.evaluate, which supports generators.
100/100 [==============================] - 3s 30ms/step - loss: 0.3397 - recall: 0.6154 - precision: 0.9233 - dice_coef: 0.7463 - mean_io_u: 0.7287 - miou_coef: 0.6603


100%|██████████| 100/100 [00:48<00:00,  2.08it/s]


In [ ]:
# %pycat run_modified.py
# %%writefile run_modified.py
!python3 run_modified.py

2020-10-17 07:18:20.721286: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
train steps: 3100 valid_steps 12
2020-10-17 07:18:25.260880: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcuda.so.1
2020-10-17 07:18:25.295058: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:982] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2020-10-17 07:18:25.295745: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1716] Found device 0 with properties: 
pciBusID: 0000:00:04.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2020-10-17 07:18:25.295800: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1
2020-10-17 07:18:25.502955

In [ ]:
# %pycat infer.py
# %%writefile infer.py
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import CustomObjectScope
from data_generator import *
from metrics import dice_coef, dice_loss, miou_coef, miou_loss

def mask_to_3d(mask):
    mask = np.squeeze(mask)
    mask = [mask, mask, mask]
    mask = np.transpose(mask, (1, 2, 0))
    return mask

if __name__ == "__main__":
    model_path = "files/daspp_miou.h5"
    save_path = "result"
    test_path = "new_data/Kvasir-SEG/test/"

    image_size = 256
    batch_size = 1

    test_image_paths = glob(os.path.join(test_path, "images", "*"))
    test_mask_paths = glob(os.path.join(test_path, "masks", "*"))
    test_image_paths.sort()
    test_mask_paths.sort()

    ## Create result folder
    try:
        os.mkdir(save_path)
    except:
        pass

    ## Model
    with CustomObjectScope({'dice_loss': dice_loss, 'dice_coef': dice_coef,'miou_loss':miou_loss,'miou_coef':miou_coef}):
        model = load_model(model_path)

    ## Test
    print("Test Result: ")
    test_steps = len(test_image_paths)//batch_size
    test_gen = DataGen(image_size, test_image_paths, test_mask_paths, batch_size=batch_size)
    model.evaluate_generator(test_gen, steps=test_steps, verbose=1)

    ## Generating the result
    for i, path in tqdm(enumerate(test_image_paths), total=len(test_image_paths)):
        image = parse_image(test_image_paths[i], image_size)
        mask = parse_mask(test_mask_paths[i], image_size)

        predict_mask = model.predict(np.expand_dims(image, axis=0))[0]
        predict_mask = (predict_mask > 0.5) * 255.0

        sep_line = np.ones((image_size, 10, 3)) * 255

        mask = mask_to_3d(mask)
        predict_mask = mask_to_3d(predict_mask)

        all_images = [image * 255, sep_line, mask * 255, sep_line, predict_mask]
        cv2.imwrite(f"{save_path}/{i}.png", np.concatenate(all_images, axis=1))
# !python3 infer_modified.py
# https://colab.research.google.com/drive/1TTtNwEF4TjWQoXdvjpWky-dnXN1zFu13

Test Result: 
Instructions for updating:
Please use Model.evaluate, which supports generators.
100/100 [==============================] - 3s 28ms/step - loss: 0.3329 - recall: 0.6972 - precision: 0.8262 - dice_coef: 0.7666 - mean_io_u: 0.7529 - miou_coef: 0.6671


100%|██████████| 100/100 [00:09<00:00, 10.07it/s]


In [ ]:
# %pycat infer.py
# dsapp_miou
# 100/100 [==============================] - 3s 28ms/step - loss: 0.3329 - recall: 0.6972 - precision: 0.8262 - dice_coef: 0.7666 - mean_io_u: 0.7529 - miou_coef: 0.6671
# 100%|██████████| 100/100 [00:09<00:00, 10.07it/s]

# miou
# 100/100 [==============================] - 3s 28ms/step - loss: 0.3052 - recall: 0.6497 - precision: 0.8981 - dice_coef: 0.7835 - mean_io_u: 0.7310 - miou_coef: 0.6948
# 100%|██████████| 100/100 [00:10<00:00,  9.93it/s]

# dsapp_relu_miou
# 100/100 [==============================] - 3s 30ms/step - loss: 0.3397 - recall: 0.6154 - precision: 0.9233 - dice_coef: 0.7463 - mean_io_u: 0.7287 - miou_coef: 0.6603
# 100%|██████████| 100/100 [00:48<00:00,  2.08it/s]

In [7]:
## GFLOPs
import os
import numpy as np
import cv2
from glob import glob
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.utils import CustomObjectScope
from data_generator import *
from metrics import dice_coef, dice_loss, miou_coef, miou_loss

def get_flops(model_h5_path):
    session = tf.compat.v1.Session()
    graph = tf.compat.v1.get_default_graph()


    with graph.as_default():
        with session.as_default():
          with CustomObjectScope({'dice_loss': dice_loss, 'dice_coef': dice_coef,'miou_loss':miou_loss,'miou_coef':miou_coef}):
            model = load_model(model_h5_path)
            # model = tf.keras.models.load_model(model_h5_path)

          run_meta = tf.compat.v1.RunMetadata()
          opts = tf.compat.v1.profiler.ProfileOptionBuilder.float_operation()

          # Optional: save printed results to file
          # flops_log_path = os.path.join(tempfile.gettempdir(), 'tf_flops_log.txt')
          # opts['output'] = 'file:outfile={}'.format(flops_log_path)

          # We use the Keras session graph in the call to the profiler.
          flops = tf.compat.v1.profiler.profile(graph=graph,
                                                run_meta=run_meta, cmd='op', options=opts)

          return flops.total_float_ops

In [8]:
# model_path = "files/daspp_relu_miou.h5" get_flops = 30074652
# model_path = "files/daspp_miou.h5" get_flops = 40099550
# model_path = "files/miou.h5" get_flops = 8718068
get_flops(model_path)

Instructions for updating:
Use `tf.compat.v1.graph_util.tensor_shape_from_node_def_name`


8718068

In [22]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 256, 256, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 256, 256, 16) 448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 256, 256, 16) 64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 256, 256, 16) 0           batch_normalization[0][0]        
_______________________________________________________________________________________